In [ ]:
from flask import Flask, render_template, request, send_file
import os

app = Flask(__name__)
UPLOAD_FOLDER = "static"
app.config["UPLOAD_FOLDER"] = UPLOAD_FOLDER

def to_bin(data):
    if isinstance(data, str):
        return ''.join(format(ord(i), '08b') for i in data)
    return format(data, '08b')

def encode_text(image_path, text, output_path):
    with open(image_path, 'rb') as f:
        img_bytes = bytearray(f.read())

    binary_text = to_bin(text) + '1111111111111110'
    data_index = 0

    for i in range(54, len(img_bytes), 3):
        if data_index < len(binary_text):
            img_bytes[i] = (img_bytes[i] & ~1) | int(binary_text[data_index])
            data_index += 1

    with open(output_path, 'wb') as f:
        f.write(img_bytes)

def decode_text(image_path):
    with open(image_path, 'rb') as f:
        img_bytes = bytearray(f.read())

    binary_data = ""
    for i in range(54, len(img_bytes), 3):
        binary_data += str(img_bytes[i] & 1)

    separator = '1111111111111110'
    parts = binary_data.split(separator)
    binary_text = parts[0] if len(parts) > 1 else binary_data

    return ''.join(chr(int(binary_text[i:i+8], 2)) for i in range(0, len(binary_text), 8))

@app.route("/", methods=["GET", "POST"])
def index():
    message = ""
    download_link = None

    if request.method == "POST":
        action = request.form.get("action")
        uploaded_file = request.files["image"]
        filename = os.path.join(app.config["UPLOAD_FOLDER"], uploaded_file.filename)
        uploaded_file.save(filename)

        if action == "encode":
            secret_text = request.form.get("secret_text")
            output_path = os.path.join(app.config["UPLOAD_FOLDER"], "output.bmp")
            encode_text(filename, secret_text, output_path)
            message = "✅ Secret message encoded successfully!"
            download_link = "/static/output.bmp"

        elif action == "decode":
            decoded = decode_text(filename)
            message = f"🔓 Decoded Message: {decoded}"

    return render_template("index.html", message=message, download_link=download_link)

if __name__ == "__main__":
    app.run(debug=True)
